In [1]:
%load_ext nb_black
%matplotlib widget
import copy
import datetime
import json
import os
import scipy.spatial
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload

import celeri
celeri = reload(celeri)

<IPython.core.display.Javascript object>

In [2]:
RUN_NAME = datetime.datetime.now().strftime("%y%m%d%H%M%S") + os.sep
with open('./data/western_north_america/command.json', 'r') as f:
    command = json.load(f)
station = pd.read_csv(command["station_file_name"])
segment = pd.read_csv(command["segment_file_name"])
block = pd.read_csv(command["block_file_name"])

<IPython.core.display.Javascript object>

In [3]:
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command)

<IPython.core.display.Javascript object>

In [4]:
np_segments = np.zeros((len(segment), 2, 2))
np_segments[:, 0, 0] = segment.lon1.to_numpy()
np_segments[:, 1, 0] = segment.lon2.to_numpy()
np_segments[:, 0, 1] = segment.lat1.to_numpy()
np_segments[:, 1, 1] = segment.lat2.to_numpy()
for i in range(np_segments.shape[0] - 1):
    plt.plot(np_segments[i,:,0], np_segments[i,:,1], 'k-')
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<IPython.core.display.Javascript object>

In [29]:
# De-duplicate the vertices and build an ptr_edge array
all_vertices = np_segments.reshape((-1, 2))
tree = scipy.spatial.KDTree(all_vertices, leafsize=1000)
duplicates = tree.query_ball_point(all_vertices, 1e-8)

ptr_edges = []
dedup_vertices = []
original_to_new = dict()
for i in range(np_segments.shape[0]):
    v1_idx = duplicates[2 * i][0]
    v2_idx = duplicates[2 * i + 1][0]
    if v1_idx == 2 * i:
        original_to_new[2 * i] = len(dedup_vertices)
        dedup_vertices.append(np_segments[i][0])
    if v2_idx == 2 * i + 1:
        original_to_new[2 * i + 1] = len(dedup_vertices)
        dedup_vertices.append(np_segments[i][1])
    ptr_edges.append((original_to_new[v1_idx], original_to_new[v2_idx]))

np_dedup_vertices = np.array(dedup_vertices)

# Check that the vertices are unique up to 1e-8 now.
new_tree = scipy.spatial.KDTree(np_dedup_vertices)
np.all([v[0] == i for i,v in enumerate(new_tree.query_ball_point(np_dedup_vertices, 1e-8))])

# Build the graph
G = nx.Graph()
for e in ptr_edges:
    G.add_edge(*e)

# Check that it is planar and build the PlanarEmbedding data structure.
is_planar, planar_embedding = nx.check_planarity(G)

# Extract each polygon. 
counted_half_edges = set()
polygons = []
for component in nx.connected_components(planar_embedding):
    for v in component:
        for w in planar_embedding.neighbors_cw_order(v):
            if (v, w) not in counted_half_edges:
                polygons.append(planar_embedding.traverse_face(v, w, counted_half_edges))

print("Found " + str(len(polygons)) + " closed polygons")

# Map polygons boundaries back to 2 labesl for each block
n_vertices = np_dedup_vertices.shape[0]
n_edges = len(ptr_edges)

vertex_to_polygon = [[] for i in range(n_vertices)] 
for p_idx, p in enumerate(polygons):
    for v_idx in p:
        vertex_to_polygon[v_idx].append(p_idx)

edge_to_polygon = []
for e in ptr_edges:
    polys_v0 = vertex_to_polygon[e[0]]
    polys_v1 = vertex_to_polygon[e[1]]
    # Intersection of lists
    polys_edge = [p_idx for p_idx in polys_v0 if p_idx in polys_v1]
    edge_to_polygon.append(polys_edge)

segment["east_label"] = np.zeros_like(segment.lon1)
segment["west_label"] = np.zeros_like(segment.lon1)
for i in range(len(segment)):
    segment.east_label.values[i] = edge_to_polygon[i][0]
    segment.west_label.values[i] = edge_to_polygon[i][1]

Found 31 closed polygons


<IPython.core.display.Javascript object>

In [32]:
# Plot one polygon and with midpoints and labels
plt.figure()
for p in polygons[2:3]:
    verts_loop = np.concatenate([np_dedup_vertices[p], np_dedup_vertices[p[0]][None,:]])
    plt.plot(verts_loop[:, 0], verts_loop[:, 1])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
edge_to_polygon[0, 1]

TypeError: list indices must be integers or slices, not tuple

<IPython.core.display.Javascript object>